In [4]:
file_path = "mpesa-statement.pdf"
raw_output =  "mpesa-statement_raw.xlsx"
formatted_output =  "mpesa-statement_formatted.xlsx"
password = '1234567' #One time code received from Safaricom

In [6]:
#Raw Output Conversion
import camelot
import pandas as pd
tables = camelot.read_pdf(file_path, flavor='stream', pages='all',password=password)
# Combine tables into a single DataFrame
df_list = [table.df for table in tables]
df = pd.concat(df_list, ignore_index=True)

# Save the DataFrame to Excel
df.to_excel(raw_output, index=False, engine='openpyxl')
df.to_csv(raw_output, index=False, engine='openpyxl')

print(f"Data extracted from PDF and saved to {raw_output}")

Data extracted from PDF and saved to mpesa-statement_raw.xlsx


In [ ]:
# Format Raw Output File to Pandas Dataframe and export to Excel
# Exclude Empty Rows
# Exclude Disclaimers

import csv
import json
import pandas as pd

# Read CSV data from the file
csv_data = ""
with open(raw_output.replace("xlsx","csv"), 'r') as file:
    print("Reading file")
    csv_data = file.read()

# Split the CSV data into rows
rows = csv_data.split('\n')

# Create a list to hold the JSON objects
data_list = []

#Initialize variables to store current receipt details
current_receipt = None
current_details = []

# Iterate through the rows and convert to JSON
start_row = 3
r = 0
for row in csv.reader(rows, delimiter=','):
    if r < start_row or len(row) == 0 or row[0] == "Receipt No." or 'Disclaimer' in row[0] or \
        'for which it was provided.' in row[0]:
        r =r+1
        continue
    
    if row[0]:  # Receipt No. column is not empty
        # If there is already a current receipt, save it to the list
        if current_receipt:
            data_list.append({
                'Receipt No.': current_receipt,
                'Completion Time': completion_time,
                'Details': '\n'.join(current_details),
                'Transaction Status': transaction_status,
                'Paid In': paid_in,
                'Withdrawn': withdrawn,
                'Balance': balance
            })
        # Update current receipt and reset details
        current_receipt = row[0]
        completion_time = row[1]
        transaction_status = row[3]
        paid_in = row[4]
        withdrawn = row[5]
        balance = row[6]
        current_details = [row[2]]
    else:
        # Append the details to the current receipt
        current_details.append(row[2])

# Add the last receipt to the list
if current_receipt:
    data_list.append({
        'Receipt No.': current_receipt,
        'Completion Time': completion_time,
        'Details': '\n'.join(current_details),
        'Transaction Status': transaction_status,
        'Paid In': paid_in,
        'Withdrawn': withdrawn,
        'Balance': balance
    })

# Convert the list to JSON
json_data = json.dumps(data_list, indent=2)

# Print the resulting JSON data
data_list = json.loads(json_data)

df = pd.DataFrame(data_list)
df['Paid In'] = pd.to_numeric(df['Paid In'], errors='coerce')
df['Withdrawn'] = pd.to_numeric(df['Withdrawn'], errors='coerce')
df['Balance'] = pd.to_numeric(df['Balance'], errors='coerce')
df = df.fillna(0)
df.to_excel(formatted_output,index=False)
df.head()